In [84]:
!pip install pdfplumber gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1


In [1]:
import spacy
from spacy.language import Language
from spacy.tokens import Doc, Span
from spacy.util import filter_spans
import pdfplumber

In [2]:
# Função para carregar o texto do PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    #text = text.replace("\n", " ")
    return text

In [71]:
pdf_path = 'deepseek.pdf'
# Extrair texto do PDF
text = extract_text_from_pdf(pdf_path)

In [72]:
text

'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via\nReinforcement Learning\nDeepSeek-AI\nresearch@deepseek.com\nAbstract\nWe introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.\nDeepSeek-R1-Zero,amodeltrainedvialarge-scalereinforcementlearning(RL)withoutsuper-\nvisedfine-tuning(SFT)asapreliminarystep,demonstratesremarkablereasoningcapabilities.\nThrough RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing\nreasoningbehaviors. However,itencounterschallengessuchaspoorreadability,andlanguage\nmixing. To address these issues and further enhance reasoning performance, we introduce\nDeepSeek-R1,whichincorporatesmulti-stagetrainingandcold-startdatabeforeRL.DeepSeek-\nR1achievesperformancecomparabletoOpenAI-o1-1217onreasoningtasks. Tosupportthe\nresearchcommunity,weopen-sourceDeepSeek-R1-Zero,DeepSeek-R1,andsixdensemodels\n(1.5B,7B,8B,14B,32B,70B)distilledfromDeepSeek-R1basedonQwenandLlama.\n100\n80\n60\n40\n20\n0\nAIME 2024 Code

In [73]:
text = text.replace('\n', ' ')
text

'DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via Reinforcement Learning DeepSeek-AI research@deepseek.com Abstract We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1. DeepSeek-R1-Zero,amodeltrainedvialarge-scalereinforcementlearning(RL)withoutsuper- visedfine-tuning(SFT)asapreliminarystep,demonstratesremarkablereasoningcapabilities. Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing reasoningbehaviors. However,itencounterschallengessuchaspoorreadability,andlanguage mixing. To address these issues and further enhance reasoning performance, we introduce DeepSeek-R1,whichincorporatesmulti-stagetrainingandcold-startdatabeforeRL.DeepSeek- R1achievesperformancecomparabletoOpenAI-o1-1217onreasoningtasks. Tosupportthe researchcommunity,weopen-sourceDeepSeek-R1-Zero,DeepSeek-R1,andsixdensemodels (1.5B,7B,8B,14B,32B,70B)distilledfromDeepSeek-R1basedonQwenandLlama. 100 80 60 40 20 0 AIME 2024 Codeforces GPQA Diamond M

In [74]:
import re
#text = re.sub(r"\s+", " ", text)
#text

In [75]:
#text = re.sub(r"[^\w\s\-\.\,]", "", text)  # Remove tudo que não é letra, número, espaço, hífen, ponto ou vírgula
#text

In [76]:
#text = text.lower()
#text

In [77]:
#text = re.sub(r"-\s+", "", text)  # Remove hifens no fim de linha
#text

In [78]:
nlp = spacy.blank("en")

In [79]:
@Language.component("model_name_recognizer")
def model_name_recognizer(doc):
    # Lista de possíveis nomes de modelos baseados no seu PDF
    model_names = [
        "DeepSeek-R1", "DeepSeek-R1-Zero", "DeepSeek-V3", "Qwen2.5-32B",
        "Llama-3.3-70B-Instruct", "QwQ-32B-Preview", "OpenAI-o1-1217",
        "OpenAI-o1-mini", "Claude-3.5-Sonnet", "GPT-4o"
    ]

    # Procurar por esses nomes no texto
    spans = []
    for model_name in model_names:
        start = 0
        while True:
            start = doc.text.find(model_name, start)
            if start == -1:
                break
            end = start + len(model_name)

            # Tentar criar um span; verificar se não é None
            char_span = doc.char_span(start, end)
            if char_span is not None:
                spans.append(Span(doc, char_span.start, char_span.end, label="MODEL"))

            start = end  # Continuar a busca após o modelo encontrado

    # Filtrar spans para evitar sobreposição
    filtered_spans = filter_spans(spans)

    # Adicionar as entidades encontradas ao doc
    doc.ents = list(doc.ents) + filtered_spans
    return doc

1. @Language.component("model_name_recognizer")

Isso registra a função como um componente personalizado do pipeline do spaCy.

2. Span(doc, doc.char_span(start, end).start, doc.char_span(start, end).end, label="MODEL")

  - Aqui estamos criando spans para as entidades encontradas no texto.

  - doc.char_span(start, end) é um método do spaCy que mapeia índices de caracteres para tokens no Doc.
  - Span(doc, start, end, label="MODEL") cria uma entidade nomeada personalizada.

3. doc.ents = list(doc.ents) + spans

Adicionamos as novas entidades detectadas ao documento.

In [80]:
# Adicionar o componente personalizado ao pipeline
nlp.add_pipe("model_name_recognizer", last=True)

<function __main__.model_name_recognizer(doc)>

In [81]:
# Processar o texto com o spaCy
doc = nlp(text)

In [82]:
# Exibir as entidades encontradas
for ent in doc.ents:
    print(ent.text, ent.label_)

DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
OpenAI-o1-1217 MODEL
DeepSeek-R1 MODEL
OpenAI-o1-mini MODEL
DeepSeek-V3 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-V3 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-V3 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
OpenAI-o1-1217 MODEL
DeepSeek-V3 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-V3 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
OpenAI-o1-mini MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-R1 MODEL
DeepSeek-R1-Zero MODEL
DeepSeek-V3 MODEL
DeepSeek-V3 MODEL
DeepSeek-V3 MODEL
DeepSeek-V3 MODEL
DeepS

In [10]:
import gradio as gr
import re

In [11]:
model_names = [
        "DeepSeek-R1", "DeepSeek-R1-Zero", "DeepSeek-V3", "Qwen2.5-32B",
        "Llama-3.3-70B-Instruct", "QwQ-32B-Preview", "OpenAI-o1-1217",
        "OpenAI-o1-mini", "Claude-3.5-Sonnet", "GPT-4o"
    ]

In [12]:
# Compilação de uma única expressão regular para todas as entidades
ENTITY_REGEX = re.compile(r"\b(" + "|".join(re.escape(name) for name in model_names) + r")\b", re.IGNORECASE)


# Função principal para processar o PDF e destacar entidades
def process_pdf(pdf_file, highlight_entities):
    # Extrair texto do PDF
    text = extract_text_from_pdf(pdf_file)

    if highlight_entities:
        # Destacar entidades usando regex
        highlighted_text = ENTITY_REGEX.sub(r"**\1**", text)
        return highlighted_text
    else:
        # Apenas exibir o texto original
        return text


```python
ENTITY_REGEX = re.compile(r"\b(" + "|".join(re.escape(name) for name in MODEL_NAMES) + r")\b", re.IGNORECASE)
```

Essa linha cria uma **expressão regular** (regex) que será usada para encontrar e destacar as entidades (nomes dos modelos, no caso) no texto. Vou explicar cada parte:

---

### **1. `MODEL_NAMES`**
Essa é a lista de entidades que queremos buscar no texto:

```python
MODEL_NAMES = [
    "DeepSeek-R1", "DeepSeek-R1-Zero", "DeepSeek-V3", "Qwen2.5-32B",
    "Llama-3.3-70B-Instruct", "QwQ-32B-Preview", "OpenAI-o1-1217",
    "OpenAI-o1-mini", "Claude-3.5-Sonnet", "GPT-4o"
]
```

Cada um desses nomes será tratado como uma entidade que queremos localizar no texto.

---

### **2. `re.escape(name)`**
A função `re.escape` é usada para garantir que qualquer caractere especial (como `-`, `.`, ou `+`) nos nomes da lista seja tratado literalmente na expressão regular.

Por exemplo:
- O nome `"DeepSeek-R1"` contém um hífen (`-`), que é um caractere especial em regex. Se não usarmos `re.escape`, o hífen poderia ser interpretado como algo diferente, causando erros ou resultados inesperados.
- Com `re.escape("DeepSeek-R1")`, o hífen é "escapado", ficando assim: `"DeepSeek\-R1"`.

Isso garante que os nomes sejam interpretados corretamente.

---

### **3. `"|".join(...)`**
A função `"|".join(...)` cria uma única string onde os nomes da lista são separados pelo caractere `|`, que em regex significa **"ou"**.

Por exemplo:
```python
"|".join(["DeepSeek-R1", "DeepSeek-V3", "GPT-4o"])
```
Produz:
```
"DeepSeek\-R1|DeepSeek\-V3|GPT\-4o"
```

Isso significa que a regex irá buscar por **qualquer um dos nomes** da lista no texto.

---

### **4. `r"\b(...)\b"`**
O `r"\b"` é um delimitador em regex que indica **limites de palavra**. Ele garante que a busca seja feita apenas por palavras completas, e não por partes de palavras.

Por exemplo:
- Se você buscar `"DeepSeek"` sem usar `\b`, ele pode coincidir com `"DeepSeek-R1"` ou até mesmo com `"DeepSeeker"`.
- Com `\bDeepSeek\b`, ele só encontrará a palavra exatamente como `"DeepSeek"`.

O trecho `r"\b(...)\b"` envolve o grupo de nomes entre limites de palavra, garantindo que os nomes sejam encontrados apenas como palavras completas.

---

### **5. `re.compile(...)`**
A função `re.compile` compila a expressão regular em um **objeto regex**, que é mais eficiente para buscas repetidas, especialmente em textos grandes. Em vez de recompilar a regex toda vez que você faz uma busca, ela é pré-compilada e pronta para uso.

---

### **6. `re.IGNORECASE`**
Esse é um **modificador** que indica que a busca será **case-insensitive**, ou seja, não fará distinção entre maiúsculas e minúsculas.

Por exemplo:
- `"DeepSeek-R1"` será encontrado mesmo que no texto esteja escrito como `"deepseek-r1"` ou `"DEEPSEEK-R1"`.

---

### **Como a Regex Final Fica?**
Depois de processar a lista `MODEL_NAMES`, a regex gerada será algo assim:

```regex
\b(DeepSeek\-R1|DeepSeek\-R1\-Zero|DeepSeek\-V3|Qwen2\.5\-32B|Llama\-3\.3\-70B\-Instruct|QwQ\-32B\-Preview|OpenAI\-o1\-1217|OpenAI\-o1\-mini|Claude\-3\.5\-Sonnet|GPT\-4o)\b
```

Essa regex:
- Busca qualquer um dos nomes da lista.
- Garante que os nomes sejam tratados como palavras completas (graças a `\b`).
- Ignora diferenças entre maiúsculas e minúsculas (graças a `re.IGNORECASE`).

---

### **Exemplo de Uso**
Se aplicarmos essa regex a um texto, ela encontra as entidades especificadas. Por exemplo:

#### Texto:
```text
O modelo DeepSeek-R1 foi comparado ao DeepSeek-V3 e ao GPT-4o.
```

#### Código:
```python
ENTITY_REGEX = re.compile(r"\b(" + "|".join(re.escape(name) for name in MODEL_NAMES) + r")\b", re.IGNORECASE)
matches = ENTITY_REGEX.findall("O modelo DeepSeek-R1 foi comparado ao DeepSeek-V3 e ao GPT-4o.")
print(matches)
```

#### Saída:
```python
['DeepSeek-R1', 'DeepSeek-V3', 'GPT-4o']
```


In [16]:
# Interface do Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Upload de PDF e Destaque de Entidades")

    with gr.Row():
        pdf_input = gr.File(label="Faça upload do PDF", file_types=[".pdf"])
        highlight_checkbox = gr.Checkbox(label="Destacar entidades (nomes dos modelos)", value=False)

    output_text = gr.Markdown(label="Texto Extraído")

    submit_button = gr.Button("Processar")
    submit_button.click(process_pdf, inputs=[pdf_input, highlight_checkbox], outputs=output_text)

# Rodar a aplicação
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://69d25595a202dd3dd4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
